# Гордеев Никита Андреевич ИУ5-64Б РК2, Вариант 4

In [125]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [126]:
data = pd.read_csv('toy_dataset.csv')
data.head()

,Number,City,Gender,Age,Income,Illness
0,1,Dallas,Male,41,40367.0,No
1,2,Dallas,Male,54,45084.0,No
2,3,Dallas,Male,42,52483.0,No
3,4,Dallas,Male,40,40941.0,No
4,5,Dallas,Male,46,50289.0,No


In [127]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Number   150000 non-null  int64  
 1   City     150000 non-null  object 
 2   Gender   150000 non-null  object 
 3   Age      150000 non-null  int64  
 4   Income   150000 non-null  float64
 5   Illness  150000 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 6.9+ MB


In [128]:
data.isnull().sum()

Number     0
City       0
Gender     0
Age        0
Income     0
Illness    0
dtype: int64

In [129]:
data = data.drop(columns=['Number'])
data['City'] = data['City'].astype('category')
data['Gender'] = data['Gender'].astype('category')
data['Illness'] = data['Illness'].astype('category')

In [130]:
TARGET_KEY = 'Illness'
X_data, y_data = data.drop(columns=[TARGET_KEY]), data[[TARGET_KEY]]

In [131]:
from sklearn.calibration import LabelEncoder


le = LabelEncoder()
y_data = le.fit_transform(y_data)
y_data

/home/codespace/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 0, 0, 0])

In [132]:
category_columns = X_data.select_dtypes(include=['category']).columns
category_columns

Index(['City', 'Gender'], dtype='object')

In [133]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
encoded_columns = ohe.fit_transform(X_data[category_columns])

In [134]:
import numpy as np


X_data[np.concatenate(ohe.categories_)] = encoded_columns.toarray()
X_data.drop(['City', 'Gender'], axis=1, inplace=True)
X_data

,Age,Income,Austin,Boston,Dallas,Los Angeles,Mountain View,New York City,San Diego,Washington D.C.,Female,Male
0,41,40367.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,54,45084.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,42,52483.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,40,40941.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,46,50289.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,48,93669.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
149996,25,96748.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
149997,26,111885.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
149998,25,111878.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [135]:

X_test, X_train, y_test, y_train = train_test_split(X_data, y_data, test_size = 0.2, random_state = 1)

In [136]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_pred

array([0.0841102 , 0.07308867, 0.08107819, ..., 0.08187744, 0.08687284,
       0.08821145])

In [137]:
gb = GradientBoostingRegressor()
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
gb_pred

array([0.08033809, 0.08043944, 0.07713108, ..., 0.0796356 , 0.08686704,
       0.08218332])

Для сравнения качества моделей используем метрики средней абсолютной ошибки и коэффициента детерминации. Первый покажет, насколько предсказанные значения отклоняются от настоящих. Второй покажет, насколько точно модель предсказывает значения в целом.

In [138]:
lr_mae = mean_absolute_error(y_test, lr_pred)
gb_mae = mean_absolute_error(y_test, gb_pred)

In [139]:
lr_r2 = r2_score(y_test, lr_pred)
gb_r2 = r2_score(y_test, gb_pred)

In [140]:
pd.DataFrame.from_dict({
    'LR': {
        'MAE': round(lr_mae, 5),
        'R2': f'{round(lr_r2 * 100, 3)}%'
    },
    'GB': {
        'MAE': round(gb_mae, 5),
        'R2': f'{round(gb_r2 * 100, 3)}%'
    }
})

,LR,GB
MAE,0.14881,0.14879
R2,-0.023%,-0.262%


По результатам можно сказать, что модели можно признать абсолютно неприемлемыми.